# Data Processing

In [25]:
using CSV, DataFrames, GLM, Statistics, Dates, Gadfly, Random, MLBase;
include("utils/precipitation.jl");

## Build features

### Get and filter the features

#### Latitude, Longitude, Height

In [2]:
features = CSV.read("data/ouvrages-surverses.csv");
colnames = ["N_Env", "ID_SOMA", "ID_OUVRAGE", "NOM", "SOMA_SEC", "REGION", "TP_X", "TP_Y", "TP_Z", "TP_LAT", "TP_LNG", "EMI_X", "EMI_Y", "EMI_LNG", "EMI_LAT"];
names!(features, Symbol.(colnames));
select!(features, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]);

#### Replace missing Z index with mean

In [3]:
features.TP_Z = coalesce.(features.TP_Z, mean(features[completecases(features), :].TP_Z));
first(shuffleDf(features), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z
,String,Float64,Float64,Float64
1,0801-09D,45.5158,-73.5357,25.28
2,4420-04D,45.4485,-73.5697,16.0
3,4340-01D,45.5262,-73.5444,19.33
4,3240-02D,45.6644,-73.5605,10.11
5,4370-05D,45.4398,-73.657,24.79
6,4230-07D,45.6888,-73.4863,9.86
7,4300-01D,45.5707,-73.5207,13.59
8,4400-02D,45.4507,-73.5724,16.17
9,4610-01D,45.4328,-73.8377,24.84


### Load dates and surverses

In [4]:
surverses = CSV.read("data/surverses.csv",missingstring="-99999");

#### Filter months

In [5]:
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);

#### Filter non rain surverses

In [6]:
raison = coalesce.(surverses[:,:RAISON],"Inconnue");
surverses[!,:RAISON] = raison;

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);

#### Remove missing data and rename

In [7]:
surverses = dropmissing(surverses, disallowmissing=true);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);
first(shuffleDf(surverses),10)

,ID_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,3250-02D,2018-09-08,0
2,3305-04D,2018-07-13,0
3,4310-01D,2017-06-11,0
4,3350-06D,2016-10-31,0
5,4230-08D,2018-07-13,0
6,3230-01D,2015-09-28,0
7,3450-01D,2016-08-16,0
8,3320-01D,2015-07-21,1
9,3260-01D,2013-06-28,1


### Augment features with dates and label

In [8]:
comb = join(features, surverses, on = :ID_OUVRAGE);
first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,SURVERSE
,String,Float64,Float64,Float64,Date,Int64
1,4770-01D,45.6574,-73.4902,10.77,2018-08-19,0
2,3580-01D,45.4085,-73.9569,25.48,2014-06-23,0
3,4350-01D,45.4991,-73.555,19.3526,2015-08-05,0
4,4270-01D,45.6105,-73.5087,11.17,2018-07-29,0
5,0801-07D,45.5191,-73.5341,33.58,2013-10-01,0
6,3490-01D,45.5067,-73.8082,23.34,2015-07-02,0
7,4270-02D,45.6112,-73.5084,19.3526,2016-06-23,0
8,4260-01D,45.6308,-73.495,12.21,2013-08-04,0
9,4230-06D,45.6821,-73.4901,8.97,2018-08-26,0


### Load precipitation data

#### Load and filter months between May & October included

In [9]:
precipitation = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitation, Symbol("St-Hubert")=>:StHubert);

precipitation = filter(row -> month(row.date) > 4, precipitation);
precipitation = filter(row -> month(row.date) < 11, precipitation); 

#### Replace missing data by 0

In [10]:
precipitation[!,:McTavish] = coalesce.(precipitation[:,:McTavish], 0);
precipitation[!,:Bellevue] = coalesce.(precipitation[:,:Bellevue], 0);
precipitation[!,:Assomption] = coalesce.(precipitation[:,:Assomption], 0);
precipitation[!,:Trudeau] = coalesce.(precipitation[:,:Trudeau], 0);
precipitation[!,:StHubert] = coalesce.(precipitation[:,:StHubert], 0);

first(shuffleDf(precipitation), 5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64,Int64
1,2014-06-27,7,0,0,0,0,0
2,2016-06-20,3,0,0,0,0,0
3,2013-05-08,8,0,0,0,0,0
4,2017-09-09,7,0,0,0,0,0
5,2014-10-12,9,0,0,0,0,0


### Extract features from precipitation

#### Sum of precipitation for the day

In [11]:
pcp_sum = by(precipitation, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum);
first(shuffleDf(pcp_sum), 5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2019-10-28,7,0,0,2,6
2,2016-08-16,768,699,641,700,618
3,2019-08-06,0,0,0,0,0
4,2013-09-04,0,0,0,0,0
5,2014-06-16,0,0,0,0,0


#### Maximum precipitation in an hour for the day

In [12]:
pcp_max = by(precipitation, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(shuffleDf(pcp_max),5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2016-08-01,0,0,0,0,0
2,2019-07-28,0,0,0,15,0
3,2014-09-27,0,0,0,0,0
4,2018-06-16,0,0,4,0,0
5,2018-05-09,0,0,9,0,0


#### Maximum precipitation during three consecutive hours in a day

In [13]:
pcp_max3h = by(precipitation, :date,  McTavish = :McTavish=>maximum3, Bellevue = :Bellevue=>maximum3, 
   Assomption = :Assomption=>maximum3, Trudeau = :Trudeau=>maximum3, StHubert = :StHubert=>maximum3)
first(shuffleDf(pcp_max3h),5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2013-07-10,187,3,80,21,0
2,2016-08-03,0,0,0,0,0
3,2015-08-04,52,0,0,30,63
4,2017-06-10,0,0,0,2,0
5,2015-07-24,2,27,20,8,0


### Add precipitation data to features

#### Get stations lat-lng

In [14]:
station_df = DataFrame(STATION = String[], LAT = Float64[], LNG = Float64[]);

push!(station_df, ["McTavish", 45.504742, -73.579167]);
push!(station_df, ["Bellevue", 45.427222, -73.929167]);
push!(station_df, ["Assomption", 45.809444, -73.434722]);
push!(station_df, ["Trudeau", 45.467778, -73.741667]);
push!(station_df, ["StHubert", 45.5175, -73.416944]);

station_df

,STATION,LAT,LNG
,String,Float64,Float64
1,McTavish,45.5047,-73.5792
2,Bellevue,45.4272,-73.9292
3,Assomption,45.8094,-73.4347
4,Trudeau,45.4678,-73.7417
5,StHubert,45.5175,-73.4169


### Standardize TP and station data

In [15]:
meanlat = mean(comb.TP_LAT);
stdlat = std(comb.TP_LAT);
comb.TP_LAT = (comb.TP_LAT .- meanlat) ./ stdlat;
station_df.LAT = (station_df.LAT .- meanlat) ./ stdlat;

meanlng = mean(comb.TP_LNG);
stdlng = std(comb.TP_LNG);
comb.TP_LNG = (comb.TP_LNG .- meanlng) ./ stdlng;
station_df.LNG = (station_df.LNG .- meanlng) ./ stdlng;

meanz = mean(comb.TP_Z);
stdz = std(comb.TP_Z);
comb.TP_Z = (comb.TP_Z .- meanz) ./ stdz;

In [16]:
station_df

,STATION,LAT,LNG
,String,Float64,Float64
1,McTavish,-0.399934,0.53979
2,Bellevue,-1.29892,-2.14237
3,Assomption,3.13364,1.64672
4,Trudeau,-0.828599,-0.705498
5,StHubert,-0.251981,1.78296


### Augment Features

#### Add pcp_sum and pcp_max columns

In [17]:
comb.PCP_SUM = zeros(size(comb, 1));
comb.PCP_MAX = zeros(size(comb, 1));
comb.PCP_MAX3 = zeros(size(comb, 1));
permutecols!(comb, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :PCP_SUM, :PCP_MAX, :PCP_MAX3, :SURVERSE]);

In [18]:
first(shuffleDf(comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,4265-01D,1.00593,1.15342,-1.21178,2016-10-23,0.0,0.0,0.0
2,3350-09D,-0.0251308,-0.486079,0.625204,2018-08-09,0.0,0.0,0.0
3,4430-06D,-1.26241,-0.121484,0.617512,2017-10-28,0.0,0.0,0.0
4,4230-06D,1.65681,1.22251,-1.6441,2016-06-17,0.0,0.0,0.0
5,3480-05D,-0.471255,-1.0262,0.799056,2017-07-11,0.0,0.0,0.0
6,4270-02D,0.834128,1.08217,-0.0467296,2013-09-03,0.0,0.0,0.0
7,4330-01D,0.0527466,0.876211,-0.845611,2014-07-03,0.0,0.0,0.0
8,4400-02D,-1.0264,0.591287,-0.536371,2013-09-09,0.0,0.0,0.0
9,4380-01D,-0.828933,0.658409,-0.0467296,2016-05-21,0.0,0.0,0.0


#### Find closest station to each ouvrage and add pcp_sum and pcp_max to it

In [19]:
for i=1:size(comb, 1)
    id_ouvrage = comb[i, 1]; 
    closest_station = "McTavish"; # initial value
    shortest_dist = -1;
    
    # Find closest station
    for j=1:size(station_df, 1)
        dist = findDistance(comb[i, :TP_LAT], comb[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);
        
        if shortest_dist == -1 || dist < shortest_dist
            shortest_dist = dist;
            closest_station = station_df[j, :STATION];
        end
    end
    
    # Augment comb with a weighted p_sum, based on the distance to the station
    p_sum = pcp_sum[∈([comb[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
#     comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist);
    comb[i, :PCP_SUM] = p_sum[1]; 
    
    # Augment comb with a weighted p_max, based on the distance to the station
    p_max = pcp_max[∈([comb[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
#     comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
    comb[i, :PCP_MAX] = p_max[1];
    
    # Augment comb with a weighted p_max3h, based on the distance to the station
    p_max3 = pcp_max3h[∈([comb[i, :DATE]]).(pcp_max3h.date), Symbol(closest_station)]
#     comb[i, :PCP_MAX3] = p_max3[1] * (1 - shortest_dist);
    comb[i, :PCP_MAX3] = p_max3[1]; 
end

#### Remove outlier in PCP_SUM and PCP_MAX AND PCP_MAX3 that cause compression

In [20]:
comb[comb[:PCP_SUM] .> 750, :PCP_SUM] = 750;
comb[comb[:PCP_MAX] .> 500, :PCP_MAX] = 500;
comb[comb[:PCP_MAX3] .> 750, :PCP_MAX3] = 750;

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[20]:1
└ @ Core In[20]:1
┌ Warning: implicit broadcasting in setindex! is deprecated; use `df[row_inds, col_ind] .= Ref(v)` broadcasting assignment to change the column in place
│   caller = setindex!(::DataFrame, ::Int64, ::BitArray{1}, ::Symbol) at deprecated.jl:1490
└ @ DataFrames /home/chaime/.julia/packages/DataFrames/yH0f6/src/deprecated.jl:1490
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[20]:2
└ @ Core In[20]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[20]:3
└ @ Core In[20]:3


In [21]:
first(shuffleDf(filter(row -> row.SURVERSE == 1, comb)), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,3390-01D,0.131398,-0.169766,0.342119,2014-06-25,0.0,0.0,0.0
2,4360-01D,-0.562793,0.757544,-0.693299,2015-08-11,537.0,117.0,272.0
3,4380-01D,-0.828933,0.658409,-0.0467296,2017-06-12,0.0,0.0,0.0
4,3350-09D,-0.0251308,-0.486079,0.625204,2017-07-31,366.0,366.0,366.0
5,4380-01D,-0.828933,0.658409,-0.0467296,2015-08-18,197.0,197.0,197.0
6,4290-01D,0.5267,1.08435,-1.19485,2017-10-15,114.0,48.0,56.0
7,3350-09D,-0.0251308,-0.486079,0.625204,2015-06-21,194.0,113.0,194.0
8,4310-01D,0.296979,0.96829,-0.880997,2013-06-11,358.0,50.0,135.0
9,4270-01D,0.826007,1.08002,-1.30563,2015-05-12,28.0,8.0,13.0


### Split dates into months and days

In [22]:
comb.MONTH = month.(comb.DATE);
comb.DAY = day.(comb.DATE);
first(shuffleDf(comb[!, [:DATE, :MONTH, :DAY]]), 5)

,DATE,MONTH,DAY
,Date,Int64,Int64
1,2015-08-24,8,24
2,2015-05-02,5,2
3,2018-08-17,8,17
4,2013-06-10,6,10
5,2018-10-30,10,30


## Standardize the PCP and Date

In [23]:
mean_pcpsum = mean(comb.PCP_SUM);
std_pcpsum = std(comb.PCP_SUM);
comb.PCP_SUM = (comb.PCP_SUM .- mean_pcpsum) ./ std_pcpsum;

mean_pcpmax = mean(comb.PCP_MAX);
std_pcpmax = std(comb.PCP_MAX);
comb.PCP_MAX = (comb.PCP_MAX .- mean_pcpmax) ./ std_pcpmax;

mean_pcpmax3 = mean(comb.PCP_MAX3);
std_pcpmax3 = std(comb.PCP_MAX3);
comb.PCP_MAX3 = (comb.PCP_MAX3 .- mean_pcpmax3) ./ std_pcpmax3;

meanmonth = mean(comb.MONTH);
stdmonth = std(comb.MONTH);
comb.MONTH = (comb.MONTH .- meanmonth) ./ stdmonth;

meanday = mean(comb.DAY);
stdday = std(comb.DAY);
comb.DAY = (comb.DAY .- meanday) ./ stdday;

In [24]:
first(shuffleDf(filter(row -> row.SURVERSE == 1, comb)), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX
,String,Float64,Float64,Float64,Date,Float64,Float64
1,3350-07D,0.0802391,-0.325532,0.168267,2015-06-08,3.10458,1.35183
2,3400-01D,0.0493937,-0.198587,0.982139,2017-08-15,0.401367,1.22877
3,3750-01D,0.8052,0.0762605,-1.23947,2018-05-03,1.99069,1.01342
4,3350-02D,0.453235,-0.0681191,-0.0467296,2015-10-28,3.5936,1.84405
5,4380-01D,-0.828933,0.658409,-0.0467296,2018-07-06,0.0345996,0.582732
6,3270-01D,1.18312,0.417032,0.239038,2014-05-10,0.523623,0.582732
7,0672-03D,1.55381,0.838395,-1.48717,2016-08-17,-0.128408,0.213567
8,4330-01D,0.0527466,0.876211,-0.845611,2016-08-16,9.78789,6.36631
9,3350-11D,-0.074433,-0.435663,0.865212,2013-06-01,1.18924,2.67467


# Validate model

### Split train and validation sets

In [114]:
r_idx = shuffle(1:size(comb, 1));
train_ceil = floor(Int, size(r_idx, 1) * 0.8);
train_set = comb[r_idx[1:train_ceil], :];
val_set = comb[r_idx[train_ceil+1:size(r_idx, 1)], :];

### Train model on train set

#### Random Forest Params

In [33]:
names_glm = [:TP_LAT, :TP_LNG, :TP_Z, :MONTH, :DAY, :PCP_SUM, :PCP_MAX, :PCP_MAX3, :SURVERSE];

#### Build the features and labels

In [39]:
train_features = train_set[:, names_glm];

In [40]:
first(shuffleDf(train_features), 10)

,TP_LAT,TP_LNG,TP_Z,MONTH,DAY,PCP_SUM,PCP_MAX
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.238094,0.893818,-0.153282,-0.290832,0.243599,-0.264248,-0.0940701
2,0.688075,1.06834,-1.24962,0.878166,-1.11222,-0.400087,-0.401707
3,-1.18852,-0.458815,1.01137,-1.45983,1.59942,-0.0876561,0.121276
4,1.00593,1.15342,-1.21178,1.46267,-0.773265,-0.400087,-0.401707
5,-0.828933,0.658409,-0.0467296,1.46267,-0.773265,-0.291416,-0.217125
6,-0.471222,-1.08189,0.88829,-0.875331,-0.773265,0.645878,0.490441
7,0.163171,-0.158884,0.395966,-0.875331,0.243599,-0.400087,-0.401707
8,-0.778607,-1.74852,0.689822,-0.290832,1.03449,-0.400087,-0.401707
9,-0.787544,0.689553,-0.0467296,0.878166,0.0176291,0.768134,1.07495


#### Build the model N features to use is log_2(N + 1)

In [109]:
val_form = @formula(SURVERSE ~ TP_LAT + TP_LNG + TP_Z + MONTH + DAY + PCP_SUM + PCP_MAX + PCP_MAX3);
val_model = glm(val_form, train_features, Bernoulli(), LogitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Bernoulli{Float64},LogitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

SURVERSE ~ 1 + TP_LAT + TP_LNG + TP_Z + MONTH + DAY + PCP_SUM + PCP_MAX + PCP_MAX3

Coefficients:
───────────────────────────────────────────────────────────────────────────────────
                Estimate  Std. Error      z value  Pr(>|z|)   Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────────
(Intercept)  -3.82011      0.0206491  -185.001       <1e-99  -3.86058    -3.77963  
TP_LAT       -0.118511     0.0208053    -5.69618     <1e-7   -0.159289   -0.0777331
TP_LNG        0.594292     0.0270528    21.9678      <1e-99   0.541269    0.647314 
TP_Z          0.100143     0.0218156     4.59046     <1e-5    0.0573858   0.142901 
MONTH        -0.0649077    0.0169554    -3.82814     0.0001  -0.0981397  -0.0316757
DAY           0.003

### Validate model on validation set

#### Single validation

In [115]:
val_features = val_set[:, names_ft];
val_labels = val_set[!, :SURVERSE];
val_pred = GLM.predict(val_model, val_features);

#### Convert val_pred to int (Threshold 0.5)

In [111]:
threshold = 0.15;
val_pred[val_pred .>= threshold] .= 1.0;
val_pred[val_pred .< threshold] .= 0.0;
val_pred = convert(Array{Int}, trunc.(val_pred))

31876-element Array{Int64,1}:
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

#### F1 score single validation

In [112]:
r = roc(val_labels, val_pred);
f1score(r)

0.3323288850624193

#### Get best threshold for F1Score

In [190]:
niter = 10;
batch_score = 0;
batch_threshold = 0;

for i=1:niter
    # Split train and val sets
    r_idx = shuffle(1:size(comb, 1));
    train_ceil = floor(Int, size(r_idx, 1) * 0.8);
    train_set = comb[r_idx[1:train_ceil], :];
    val_set = comb[r_idx[train_ceil+1:size(r_idx, 1)], :];
    
    # Build features and labels
    train_features = train_set[:, names_glm];
    
    # Build model
    val_model = glm(val_form, train_features, Bernoulli(), LogitLink())
    
    # Validate model
    val_features = val_set[:, names_ft];
    val_labels = val_set[!, :SURVERSE];
    
    # Get best threshold
    start_threshold = 0.1;
    max_threshold = 0.15;
    step = 0.0002;
    
    best_threshold = start_threshold;
    score = -1;
    
    # Get best threshold
    for j=start_threshold:step:max_threshold
        val_pred = GLM.predict(val_model, val_features);
        val_pred[val_pred .>= j] .= 1.0;
        val_pred[val_pred .< j] .= 0.0;
        val_pred = convert(Array{Int}, trunc.(val_pred))
        
        r = roc(val_labels, val_pred);
        new_score = f1score(r);
        
        if new_score > score
            score = new_score
            best_threshold = j
        end
    end
    
    batch_score += score;
    batch_threshold += best_threshold;
end

batch_threshold = batch_threshold / niter;
batch_score = batch_score / niter

0.35832385421615476

In [191]:
batch_threshold

0.10091999999999998

# Submission model creation

### Separate features and labels

In [148]:
full_train_features = comb[:, names_glm];

In [149]:
full_train_labels = comb[:, :SURVERSE];

### Build Model

In [155]:
model = glm(val_form, full_train_features, Bernoulli(), LogitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Bernoulli{Float64},LogitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

SURVERSE ~ 1 + TP_LAT + TP_LNG + TP_Z + MONTH + DAY + PCP_SUM + PCP_MAX + PCP_MAX3

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error     z value  Pr(>|z|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)  -3.83333     0.0186123  -205.957      <1e-99  -3.86981    -3.79685  
TP_LAT       -0.116567    0.0186532    -6.2492     <1e-9   -0.153127   -0.0800078
TP_LNG        0.605958    0.0243601    24.875      <1e-99   0.558213    0.653703 
TP_Z          0.104758    0.0195648     5.35443    <1e-7    0.0664121   0.143105 
MONTH        -0.075803    0.0152066    -4.98487    <1e-6   -0.105607   -0.0459986
DAY           0.0167508   0.015349 

# Prediction

## Get the test data

In [156]:
test = CSV.read("data/test.csv");
rename!(test, :NO_OUVRAGE => :ID_OUVRAGE);
first(test, 10)

,ID_OUVRAGE,DATE
,String,Date
1,3260-01D,2019-05-02
2,3260-01D,2019-05-09
3,3260-01D,2019-05-10
4,3260-01D,2019-05-15
5,3260-01D,2019-05-20
6,3260-01D,2019-05-23
7,3260-01D,2019-05-24
8,3260-01D,2019-05-26
9,3260-01D,2019-05-30


In [177]:
to_merge = unique(comb[!, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]], :ID_OUVRAGE);
test_comb = join(test, to_merge, on= [:ID_OUVRAGE]);
nrow(test_comb)

283

In [178]:
first(shuffleDf(test_comb), 10)

,ID_OUVRAGE,DATE,TP_LAT,TP_LNG,TP_Z
,String,Date,Float64,Float64,Float64
1,4350-01D,2019-05-12,-0.465715,0.725081,-0.0467296
2,4380-01D,2019-08-02,-0.828933,0.658409,-0.0467296
3,3350-07D,2019-07-03,0.0802391,-0.325532,0.168267
4,4350-01D,2019-08-04,-0.465715,0.725081,-0.0467296
5,3350-07D,2019-06-15,0.0802391,-0.325532,0.168267
6,3350-07D,2019-09-30,0.0802391,-0.325532,0.168267
7,4380-01D,2019-09-02,-0.828933,0.658409,-0.0467296
8,3350-07D,2019-06-16,0.0802391,-0.325532,0.168267
9,3350-07D,2019-06-17,0.0802391,-0.325532,0.168267


### Add PCP_SUM and PCP_MAX

#### Initialize default pcp

In [179]:
test_comb.PCP_SUM = zeros(size(test_comb, 1));
test_comb.PCP_MAX = zeros(size(test_comb, 1));
test_comb.PCP_MAX3 = zeros(size(test_comb, 1));
permutecols!(test_comb, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :PCP_SUM, :PCP_MAX, :PCP_MAX3]);

In [180]:
first(shuffleDf(test_comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,3350-07D,0.0802391,-0.325532,0.168267,2019-09-30,0.0,0.0,0.0
2,3350-07D,0.0802391,-0.325532,0.168267,2019-09-24,0.0,0.0,0.0
3,4350-01D,-0.465715,0.725081,-0.0467296,2019-05-22,0.0,0.0,0.0
4,3260-01D,1.29293,0.531312,0.0790333,2019-07-14,0.0,0.0,0.0
5,4380-01D,-0.828933,0.658409,-0.0467296,2019-05-18,0.0,0.0,0.0
6,4350-01D,-0.465715,0.725081,-0.0467296,2019-06-14,0.0,0.0,0.0
7,4350-01D,-0.465715,0.725081,-0.0467296,2019-08-04,0.0,0.0,0.0
8,4350-01D,-0.465715,0.725081,-0.0467296,2019-09-18,0.0,0.0,0.0
9,3350-07D,0.0802391,-0.325532,0.168267,2019-06-16,0.0,0.0,0.0


#### Populate pcp

In [181]:
for i=1:size(test_comb, 1)
    id_ouvrage = test_comb[i, 1]; 
    closest_station = "McTavish"; # initial value
    shortest_dist = -1;
    
    # Find closest station
    for j=1:size(station_df, 1)
        dist = findDistance(test_comb[i, :TP_LAT], test_comb[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);
        
        if shortest_dist == -1 || dist < shortest_dist
            shortest_dist = dist;
            closest_station = station_df[j, :STATION];
        end
    end
    
    # Augment comb with a weighted p_sum, based on the distance to the station
    p_sum = pcp_sum[∈([test_comb[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
#     test_comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist); 
    test_comb[i, :PCP_SUM] = p_sum[1]; 
    # Augment comb with a weighted p_max, based on the distance to the station
    p_max = pcp_max[∈([test_comb[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
#     test_comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
    test_comb[i, :PCP_MAX] = p_max[1];
    # Augment comb with a weighted p_max3, based on the distance to the station
    p_max3 = pcp_max3h[∈([test_comb[i, :DATE]]).(pcp_max3h.date), Symbol(closest_station)]
#     test_comb[i, :PCP_MAX3] = p_max3[1] * (1 - shortest_dist);
    test_comb[i, :PCP_MAX3] = p_max3[1];
end

In [182]:
first(shuffleDf(test_comb), 10)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX,PCP_MAX3
,String,Float64,Float64,Float64,Date,Float64,Float64,Float64
1,4350-01D,-0.465715,0.725081,-0.0467296,2019-08-21,37.0,18.0,37.0
2,4350-01D,-0.465715,0.725081,-0.0467296,2019-08-23,0.0,0.0,0.0
3,4350-01D,-0.465715,0.725081,-0.0467296,2019-09-08,12.0,10.0,12.0
4,4240-01D,1.28137,1.24035,-1.19178,2019-08-28,134.0,30.0,73.0
5,4240-01D,1.28137,1.24035,-1.19178,2019-05-25,6.0,2.0,4.0
6,3350-07D,0.0802391,-0.325532,0.168267,2019-05-19,57.0,27.0,46.0
7,4350-01D,-0.465715,0.725081,-0.0467296,2019-05-11,2.0,2.0,2.0
8,3350-07D,0.0802391,-0.325532,0.168267,2019-07-08,0.0,0.0,0.0
9,4240-01D,1.28137,1.24035,-1.19178,2019-06-20,391.0,167.0,348.0


### Standardize PCP

In [183]:
test_comb.PCP_SUM = (test_comb.PCP_SUM .- mean_pcpsum) ./ std_pcpsum;
test_comb.PCP_MAX = (test_comb.PCP_MAX .- mean_pcpmax) ./ std_pcpmax;
test_comb.PCP_MAX3 = (test_comb.PCP_MAX3 .- mean_pcpmax3) ./ std_pcpmax3;

In [184]:
first(test_comb, 20)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,DATE,PCP_SUM,PCP_MAX
,String,Float64,Float64,Float64,Date,Float64,Float64
1,3260-01D,1.29293,0.531312,0.0790333,2019-05-02,-0.0469042,-0.0325426
2,3260-01D,1.29293,0.531312,0.0790333,2019-05-09,0.808886,0.982661
3,3260-01D,1.29293,0.531312,0.0790333,2019-05-10,4.82974,1.6287
4,3260-01D,1.29293,0.531312,0.0790333,2019-05-15,-0.37292,-0.34018
5,3260-01D,1.29293,0.531312,0.0790333,2019-05-20,0.224775,0.828842
6,3260-01D,1.29293,0.531312,0.0790333,2019-05-23,1.97711,1.53641
7,3260-01D,1.29293,0.531312,0.0790333,2019-05-24,-0.223496,-0.124834
8,3260-01D,1.29293,0.531312,0.0790333,2019-05-26,-0.359336,-0.309416
9,3260-01D,1.29293,0.531312,0.0790333,2019-05-30,-0.305,-0.186361


#### Split dates into month and day

In [185]:
test_comb.MONTH = month.(test_comb.DATE);
test_comb.DAY = day.(test_comb.DATE);

first(shuffleDf(test_comb[!, [:DATE, :MONTH, :DAY]]), 5)

,DATE,MONTH,DAY
,Date,Int64,Int64
1,2019-09-13,9,13
2,2019-09-07,9,7
3,2019-08-18,8,18
4,2019-05-23,5,23
5,2019-08-17,8,17


#### Standardize months and days

In [186]:
test_comb.MONTH = (test_comb.MONTH .- meanmonth) ./ stdmonth;
test_comb.DAY = (test_comb.DAY .- meanday) ./ stdday;

In [187]:
first(shuffleDf(test_comb[!, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :MONTH, :DAY, :PCP_SUM, :PCP_MAX, :PCP_MAX3]]), 5)

,ID_OUVRAGE,TP_LAT,TP_LNG,TP_Z,MONTH,DAY,PCP_SUM
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,3350-07D,0.0802391,-0.325532,0.168267,-1.45983,0.356584,0.374199
2,3350-07D,0.0802391,-0.325532,0.168267,-0.875331,-0.0953558,0.319863
3,4350-01D,-0.465715,0.725081,-0.0467296,-1.45983,1.59942,-0.305
4,4380-01D,-0.828933,0.658409,-0.0467296,-0.290832,0.356584,-0.223496
5,4240-01D,1.28137,1.24035,-1.19178,-0.290832,-0.547295,5.08784


### Create Test features

In [188]:
test_comb.SURVERSE = zeros(size(test_comb, 1));
test_features = test_comb[:, names_glm]

,TP_LAT,TP_LNG,TP_Z,MONTH,DAY,PCP_SUM,PCP_MAX,PCP_MAX3
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.29293,0.531312,0.0790333,-1.45983,-1.56416,-0.0469042,-0.0325426,0.0430963
2,1.29293,0.531312,0.0790333,-1.45983,-0.773265,0.808886,0.982661,1.41166
3,1.29293,0.531312,0.0790333,-1.45983,-0.66028,4.82974,1.6287,2.75979
4,1.29293,0.531312,0.0790333,-1.45983,-0.0953558,-0.37292,-0.34018,-0.365429
5,1.29293,0.531312,0.0790333,-1.45983,0.469568,0.224775,0.828842,0.492474
6,1.29293,0.531312,0.0790333,-1.45983,0.808523,1.97711,1.53641,2.51467
7,1.29293,0.531312,0.0790333,-1.45983,0.921508,-0.223496,-0.124834,-0.181592
8,1.29293,0.531312,0.0790333,-1.45983,1.14748,-0.359336,-0.309416,-0.345002
9,1.29293,0.531312,0.0790333,-1.45983,1.59942,-0.305,-0.186361,-0.263297


## Predict

In [192]:
test_labels = GLM.predict(val_model, test_features);
test_labels[test_labels .>= batch_threshold] .= 1.0;
test_labels[test_labels .< batch_threshold] .= 0.0;
test_labels = convert(Array{Int}, trunc.(test_labels))

283-element Array{Int64,1}:
 0
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0

## Generate submission

In [193]:
ID = test_comb[:,:ID_OUVRAGE].*"_".*string.(test_comb[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=test_labels)
CSV.write("submissions/mc-submission-12.csv",sampleSubmission)

"submissions/mc-submission-12.csv"